# P-value and effect size basics

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import scipy.stats as sst

from IPython.display import Image as Image
from IPython.display import display as display


## Create a normal distribution

In [ ]:
# Let's fix the seed of the random generator
# choose a number :)
np.random.seed(42)

In [ ]:
# define the normal 0,1 object
norm01 = sst.norm(0,1)
sample_size = (30,)
sample = norm01.rvs(size=sample_size)


In [ ]:
# compute mean and corrected standard deviation
sample_mean = sample.mean()
#sample_std_biased = sample.std()
sample_std = np.sqrt(np.var(sample, ddof=1))
# print(sample_std,sample_std_biased)
N = len(sample)

print("sample mean: ",sample_mean, "sample_std: ", sample_std)

In [ ]:
# np.var?

## compute the t-value and corresponding p-value

We have to estimate the effect $\mu$ and the standard deviation $\sigma_{\mu}$, the t statistic would be:

$$
t = \frac{\hat{\mu}}{\hat{\sigma_{\mu}}} 
$$

Where $\hat{\sigma_{\mu}}$ is estimated with the standard deviation of the data Y and the number of sample $n$ : $\hat{\sigma_{Y}}/\sqrt{n}$ 

In [ ]:
t_value = sample_mean / (sample_std/np.sqrt(N))
print(t_value)

In [ ]:
def t_value_from_sample(sample):
    """
    Take a sample of data (numpy array) return t-value
    """
    
    N = len(sample)
    sample_mean = sample.mean()
    sample_std = np.sqrt(np.var(sample, ddof=1))
    t_val = sample_mean / (sample_std/np.sqrt(N))
    return t_val

In [ ]:
t_val = t_value_from_sample(sample)
print(t_val)

### Use the "survival function" to go from t value to p-value

In [ ]:
N = len(sample)
p_val = sst.t.sf(t_val, df=N-1)
print(t_val, p_val)

In [ ]:
assert p_val == 1 - sst.t.cdf(t_value_from_sample(sample), df=N-1)

### Use inverse survival function to go from p-value to t-value

In [ ]:
print("t-value: ", sst.t.isf(p_val, df=N-1))

## Do this for a number of experiments

In [ ]:
sample_size = 16
distrib = sst.norm(.22, 1)
nb_of_experiments = 20
alpha = 0.05

effect_size = []

for idx in range(nb_of_experiments):
    sample =  distrib.rvs(size=(sample_size,))
    t_val = t_value_from_sample(sample)
    p_val = sst.t.sf(t_value_from_sample(sample), df=sample_size-1)
    signif = int(p_val <= alpha)
    print(t_val, "\t\t", p_val,"\t",  "*"*signif, )
    if signif:
        effect_size.append(sample.mean())

print('\n Effect_size corresponding to "significant" p-values: \n', effect_size) 

In [ ]:
print(np.asanyarray(effect_size).mean(), np.asanyarray(effect_size).std())

## Relate this to bias and file drawer effect

## Quick effect size reminder

### What is the effect ?


$$\hspace{3cm}\mu = \mu_1 - \mu_2$$

### What is the standardized effect ? (eg Cohen's d)

$$\hspace{3cm}d = \frac{\mu_1 - \mu_2}{\sigma} = \frac{\mu}{\sigma}$$

### "Z" : Effect accounting for the sample size 

$$\hspace{3cm}Z = \frac{\mu}{\sigma / \sqrt{n}}$$

### Cohen's d value:

In [ ]:
# print some cohen values
# %pylab inline
muse = (.05, .1,.2,.3,.4,.5);
sigmas = np.linspace(1.,.5,len(muse))
cohenstr = ["For sigma = %3.2f and m = %3.2f Cohen d = %3.2f" %(sig,mu,coh) 
       for (sig,mu,coh) in zip(sigmas,muse, np.asarray(muse)/sigmas)]
for s in cohenstr:
    print(s)

## Confidence intervals - file drawer

$$ P\left(\bigg|\frac{\bar{Y} - \mu}{\hat\sigma/\sqrt{n}}\bigg| \leq t_{0.025}\right) = 0.05 $$ 

$$ P\left( -t_{0.025}\hat\sigma/\sqrt{n} + \bar{Y}  \leq \mu \leq t_{0.025}\hat\sigma/\sqrt{n} + \bar{Y}   \right) = 0.05 $$ 

In [ ]:
def confidence_interval(CI = .95, **prmtrs):
    """
    inputs
    ----------
    CI: confidence interval
    prmtrs: a dictionary with our parameters, 
        example: prmtrs = {'n':16, 'mu':.3, 'sigma': 1., 'alpha': 0.05}
        
    returns:
    --------
    effect: the estimated effect
    detect: an array of 0 or 1, 1 when the effect is detected at alpha
    lCI: lower bound of confidence interval
    uCI: upper bound of confidence interval
    """
    # unpack parameters:
    n = prmtrs['n']
    mu = prmtrs['mu']; 
    alpha = prmtrs['alpha']; 
    sigma = prmtrs['sigma']
    df = n-1
    theta = mu*np.sqrt(n)/sigma
    
    # compute random variables and thresholds
    norv = sst.norm(0., sigma)
    strv = sst.t(df)
    # get the 0.05 t value *under the null* to construct confidence interval
    
    t_ci = strv.isf((1-CI)/2)
    # get the alpha level t value *under the null* to detect 
    t_alph = strv.isf(alpha)

    sample = norv.rvs(size=(n,)) + mu
    # effect and normalized effect size
    effect = sample.mean()
    # np.std takes ddof as the df of freedom lost, here: 1.
    std_error_data = np.std(sample, ddof=1) 
    std_error_mean = std_error_data/np.sqrt(n) 
    t = effect/std_error_mean
    # confidence interval :
    CI_ = t_ci*std_error_mean
    lCI = effect - CI_
    uCI = effect + CI_ 

    return (effect, lCI, uCI, t)

In [ ]:
#---------------------- parameters ------------------#
prmtrs = {'n':10, 'mu': .30, 'sigma': 1., 'alpha': 0.05}
# effect size:
theta = prmtrs['mu']*np.sqrt(prmtrs['n'])/prmtrs['sigma']
print('Theta value {:.3f} \n'.format(theta))

#--------------  simulate Nexp experiments ---------#
Nexp = 1000; 
effect = np.zeros((Nexp,))
lCI = np.zeros((Nexp,))
uCI = np.zeros((Nexp,))
t = np.zeros((Nexp,))
for i in range(Nexp):    
    effect[i], lCI[i], uCI[i], t[i] = \
                                confidence_interval(CI=.95, **prmtrs)

print('Average t {:.3f} \n'.format(t.mean()))
t_alpha = sst.t(prmtrs['n']-1).isf(alpha)
print("t_alpha", t_alpha)
detect = t>t_alpha
print("number of detection:", detect.sum())

In [ ]:

print("Mean effect {:.3f} compared to average detected effect {:.3f} \n".format(
                    effect.mean(), effect[detect].mean()))

print("\n-------------- on detections")

print("-- # of exp. where lower bound > mu: {}".format((lCI[detect]>prmtrs['mu']).sum()))
print("-- # of exp. where upper bound < mu: {}".format((uCI[detect]<prmtrs['mu']).sum()))
not_in_CI = (uCI[detect]<prmtrs['mu']).sum() + (lCI[detect]>prmtrs['mu']).sum()
print("-- Not in CI = {:d}".format(not_in_CI))
print("-- {} detections".format(detect.sum()))
print("-- percentage = {:.3f}".format(not_in_CI/detect.sum()))


print("\n-------------- all experiment, not only detected")

print("-- # of exp. where lower bound > mu: {}".format((lCI>prmtrs['mu']).sum()))
print("-- # of exp. where upper bound < mu: {}".format((uCI<prmtrs['mu']).sum()))
print("-- over {} experiments".format(Nexp))
print("-- percentage = {:.3f}".format(
      ((lCI>prmtrs['mu']).sum() + (uCI<prmtrs['mu']).sum())/Nexp))

In [ ]:
#--------------  plot ------------------------------#
mu = prmtrs['mu']
x = np.arange(Nexp)
xd = np.arange(detect.sum())
mu_line = np.ones((Nexp,))*prmtrs['mu']

# print the number of lower confidence interval values that are above the true mean:
# this should be about the risk of error/2
print("lCI > mu :  {:.3}, compare with {:.3} ".format( 
                (lCI > mu).sum() / (1.*detect.sum()),  prmtrs['alpha'])) #
print(Nexp)
# there should be none of these:
# print "(lCI < 0 ", (lCI[detect] < 0).sum() / detect.sum()

f = plt.figure(1).set_size_inches(12,4)
lines = plt.plot(xd, lCI[detect], 'g-', 
                 xd, effect[detect], 'b--',
                 xd, uCI[detect], 'r-',
                 xd, mu_line[detect], 'k');
plt.legend( lines, ('lower_bound','detected Effect', 'Upper bound', 'True effect'), 
                   loc='upper right', shadow=True)
plt.xlabel(" One x is one experiment where detection occured", fontdict={'size':14})
plt.ylabel(" Effect value and confidence interval ", fontdict={'size':14})
plt.title(" Detected effects and their confidence interval", fontdict={'size':16});

## What happens if ...

### Type I error is stricter 


### sample size goes down / up